# Preenchimento reativo
Mostra o comportamento da rede utilizando o preenchimento reativo. Essa simulação registra informações das ``requests``:

    - ID da requisição 
    - Alice
    - Bob
    - Fidelidade Mínima
    - Número de EPRs
    - Início (Quando começou a ser tratada)
    - Término (Quando foi atendida)
    - Novo Registro (Se foi necessário criar uma regra para atender, ou se já foi registrado)

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request, clear_file
from quantumnet.objects import time

Informações para a simulação

In [2]:
# Caminho do arquivo de saída
path = "resultados/reativo/sync.csv"
# Limpa o arquivo de saída
clear_file(path)
# Sobre a rede
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
row, col = 3, 4
rede.set_ready_topology("Grade", (row, col))
# Sobre o controlador
controlador = Controller(rede)
controlador.default_ttl = 100
# Sobre as requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts), fmin_range, neprs_range) for i in range(num_requests))

Funções responsáveis pela passagem de tempo

In [3]:
def update_time(time_for_update):
    """
    Atualiza o tempo e as regras

    Args:
        time_for_update (int): Tempo para incrementar o time-slot.
    """
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        # Atualiza os recursos de 10 em 10
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")

Preenchimento reativo, baseado nas requests.

    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.

In [4]:
for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            update_time(3)
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time() 
            # Registra no CSV como um novo registro
            register_request(request, "Novo_Registro", path)
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            update_time(1)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            # Registra no CSV como já registrado
            register_request(request, "Ja_Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

[Time 0] Processando requisição R-3EC980...
Decisão aplicada: (High Fidelity,)
Request R-3EC980: Start Time = 0, End Time = 3
[Time 3] Processando requisição R-BA6850...
Decisão aplicada: (High Fidelity,)
Request R-BA6850: Start Time = 3, End Time = 6
[Time 6] Processando requisição R-2EE850...
Decisão aplicada: (High Fidelity,)
Request R-2EE850: Start Time = 6, End Time = 9
[Time 9] Processando requisição R-2FAFD0...
[Time 10] Recursos atualizados
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
[BasicRule] Passo: 2
[BasicRule] Ação: Swap(5-6-7)
Running Swap(5-6-7)
Request R-2FAFD0: Start Time = 9, End Time = 12
[Time 12] Processando requisição R-2FB100...
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
[BasicRule] Passo: 2
[BasicRule] Ação: Swap(5-4-8)
Running Swap(5-4-8)
Request R-2FB100: Start Time = 12, End Time = 15
[Time 15] Processando requisição R-2F3530...
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
[BasicRule] Passo: 2
[BasicRule] Ação: Swap(9-5-7)
Running 

Registro de recursos

In [5]:
rede.registry_of_resources

{'qubits created': 3480,
 'qubits used': 0,
 'eprs created': 10880,
 'eprs used': 0}

Tabelas ao final

In [6]:
hosts = rede.hosts
for alice in hosts:
    print(f"Regras de {alice}")
    rede.get_host(alice).draw_flow_table()

Regras de 0
+-------------------------------------+-------------+-----------+
| Match                               | Route       | Rule      |
+=====================================+=============+===========+
| (10, (0.79, 0.99), 2)               | [0, 11, 10] | BasicRule |
+-------------------------------------+-------------+-----------+
| (3, (0.43000000000000005, 0.63), 4) | [0, 3]      | BasicRule |
+-------------------------------------+-------------+-----------+
| (6, (0.6900000000000001, 0.89), 2)  | [0, 5, 6]   | BasicRule |
+-------------------------------------+-------------+-----------+
Regras de 1
+-----------------------------------+---------+-------------------+
| Match                             | Route   | Rule              |
+===================================+=========+===================+
| (8, (0.55, 0.75), 2)              | [1, 8]  | BasicRule         |
+-----------------------------------+---------+-------------------+
| (3, (0.7000000000000001, 0.9), 4) | [1, 